In [3]:
import pandas as pd
import nltk

from yargy.interpretation import fact
from yargy import rule, Parser, or_, and_, not_
from yargy.pipelines import morph_pipeline
from yargy.predicates import gram

In [4]:
FILE_NAME = 'test_data.csv'

# Заготовки для реплик привествия и прощания

In [5]:
HELLO_DATASET = {'examples': ['доброго времени суток', 'приветствую тебя', 'доброе утро', 'вечер добрый', 'привет', 'с добрым утром', 'день добрый', 'приветствую вас', 'добрый день', 'добрый вечер', 'доброй ночи', 'здравствуй', 'приветики', 'приветствую', 'здравствуйте']}

In [6]:
BYE_DATASET = {'examples': ['счастливо', 'всего наилучшего', 'до вечера', 'до завтра', 'всего доброго', 'до скорых встреч', 'досвидания', 'прощайте', 'до свидания']}

# Настройки парсера для извлечения реплик приветсвия и прощания

In [7]:
HelloOrBy = fact(
    'HelloOrBy',
    ['hello', 'bye']
)


HELLO = morph_pipeline(HELLO_DATASET['examples']).interpretation(HelloOrBy.hello)
BYE = morph_pipeline(BYE_DATASET['examples']).interpretation(HelloOrBy.bye)


HELLO_BYE_PARSER = Parser(or_(HELLO, BYE).interpretation(HelloOrBy))

# Настройки парсера для извлечения названия компании

In [8]:
ORGFORM = morph_pipeline([
    'компания'
])


ORGABBR = morph_pipeline([
    'ооо',
    'оао',
    'зао',
    'ип',
    'ао'
])

ADJF = and_(gram('ADJF'), not_(gram('Anph')))
NOUN_AND_NOT_VERB = and_(gram('NOUN'), not_(gram('VERB')))


ORGNAME = or_(
    NOUN_AND_NOT_VERB,
    ADJF
)


ORGANIZATION = or_( 
    rule(ORGFORM, ORGNAME.repeatable()),
    rule(ORGABBR, ORGNAME.repeatable()),
)

ORG_PARSER = Parser(ORGANIZATION)

# Настройки парсера для извлечения реплики где менеджер представился

In [9]:
Introduce = fact(
    'Introduce',
    ['NameSurnPatr']
)

NameSurnPatr = fact(
    'NameSurnPatr',
    ['name', 'surname', 'patronymic']
)


NAME = gram('Name').interpretation(NameSurnPatr.name)
SURN = gram('Surn').interpretation(NameSurnPatr.surname)
PATR = gram('Patr').interpretation(NameSurnPatr.patronymic)


NAME_SURN_PATR = or_(
    rule(NAME).interpretation(NameSurnPatr),
    rule(NAME, PATR).interpretation(NameSurnPatr),
    rule(NAME, SURN).interpretation(NameSurnPatr),
    rule(NAME, SURN, PATR).interpretation(NameSurnPatr),
    rule(NAME, PATR, SURN).interpretation(NameSurnPatr),
    rule(SURN, NAME).interpretation(NameSurnPatr), 
    rule(SURN, NAME, PATR).interpretation(NameSurnPatr), 
).interpretation(
    Introduce.NameSurnPatr
)


THIS = morph_pipeline(['это'])
I = morph_pipeline(['я', 'мой'])
PRESERNT = morph_pipeline(['представиться'])

NAME_IS = morph_pipeline([
    'зовут',
    'имя'
])


INTRODUCE = or_(
    rule(I, NAME_SURN_PATR).interpretation(Introduce),
    rule(THIS, NAME_SURN_PATR).interpretation(Introduce),
    rule(PRESERNT, NAME_SURN_PATR).interpretation(Introduce),
    rule(I, NAME_IS, NAME_SURN_PATR).interpretation(Introduce),
    rule(NAME_IS, I, NAME_SURN_PATR).interpretation(Introduce),
    rule(I, NAME_SURN_PATR, NAME_IS).interpretation(Introduce),
    rule(NAME_IS, NAME_SURN_PATR, I).interpretation(Introduce)
).interpretation(
    Introduce
)

INTRODUCE_PARSER = Parser(INTRODUCE)

# Анализ

In [10]:
COL_COMPANY_NAME = 'company_name'
COL_NAME = 'name'
COL_SURNAME = 'surname'
COL_PATRONYMIC = 'patronymic'
COL_INTRODUCE = 'introduce'
COL_GREETING = 'greeting'
COL_BYE = 'bye'
SEPARATOR_FOR_VALUES = ' AND '

In [11]:
df = pd.read_csv(FILE_NAME)

df['text'] = df["text"].apply(lambda x: x.lower())
 
df.insert(len(df.columns), COL_COMPANY_NAME, [None for i in range(len(df))])
df.insert(len(df.columns), COL_NAME, [None for i in range(len(df))])
df.insert(len(df.columns), COL_SURNAME, [None for i in range(len(df))])
df.insert(len(df.columns), COL_PATRONYMIC, [None for i in range(len(df))])
df.insert(len(df.columns), COL_INTRODUCE, [False for i in range(len(df))])
df.insert(len(df.columns), COL_GREETING, [False for i in range(len(df))])
df.insert(len(df.columns), COL_BYE, [False for i in range(len(df))])

df.head()

,dlg_id,line_n,role,text,company_name,name,surname,patronymic,introduce,greeting,bye
0,0,0,client,алло,None,None,None,None,False,False,False
1,0,1,manager,алло здравствуйте,None,None,None,None,False,False,False
2,0,2,client,добрый день,None,None,None,None,False,False,False
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,None,None,None,None,False,False,False
4,0,4,client,ага,None,None,None,None,False,False,False


In [12]:
def find_introduce(replica):
    
    matches = list(INTRODUCE_PARSER.findall(replica))
    if len(matches) > 0:
        names = [match.fact.NameSurnPatr for match in matches]
        return names[0]
    

In [13]:
def find_orgs(replica):
    
    matches = list(ORG_PARSER.findall(replica))
    spans = [_.span for _ in matches]
    
    return spans


In [14]:
def find_greeting_or_bye(replica):
    matches = list(HELLO_BYE_PARSER.findall(replica))
    
    if len(matches) > 0:
        facts = [match.fact for match in matches]
        fact = facts[-1]
        
        if fact.bye is None:
            return 'greeting'
        else:
            return 'bye'

In [15]:
def analize_dialog(df):
    
    stop_words = set(nltk.corpus.stopwords.words("russian") + ['ага', 'угу', '+', 'та', 'то', 'э', 'aлло'])
    stop_words.remove('всего')
    stop_words.remove('до')
    
    for i in range(len(df)):
        
        replica = df.iloc[i]['text']
        
        
        # ПАРСИНГ ИМЕНИ МЕНЕДЖЕРА, ЕСЛИ ОН ПРЕДСТАВЛЯЕТСЯ В НЕЙ
        introduce_name = find_introduce(replica)
        if introduce_name is not None:
            df.loc[i, COL_NAME] = introduce_name.name
            df.loc[i, COL_SURNAME] = introduce_name.surname
            df.loc[i, COL_PATRONYMIC] = introduce_name.patronymic
            df.loc[i, COL_INTRODUCE] = True


        # УДАЛЕНИЕ ИЗ РЕПЛИКИ СТОПСЛОВ        
        splited_replica = replica.split(' ')
        replica_without_stopwords = ' '.join([word for word in splited_replica if word not in stop_words])
        
        
        if len(replica_without_stopwords) > 0:
            
            # ПРОВЕРКА ДЛЯ РЕПЛИК ПРОЩАНИЯ ИЛИ ПРИВЕТСВИЯ
            greeting_or_by = find_greeting_or_bye(replica_without_stopwords)
            if greeting_or_by is not None:
                if greeting_or_by == 'bye':
                    df.loc[i, COL_BYE] = True
                elif greeting_or_by == 'greeting':
                    df.loc[i, COL_GREETING] = True
                        
                        
            # ПАРСИНГ НАЗВАНИЯ ОРГАНИЗАЦИИ
            orgs_spans = find_orgs(replica_without_stopwords)
            if len(orgs_spans) > 0:
                df.loc[i, COL_COMPANY_NAME] = SEPARATOR_FOR_VALUES.join([replica_without_stopwords[span.start:span.stop] for span in orgs_spans])

In [16]:
analize_dialog(df)

# ПРОВЕРКА ТРЕБОВАНИЯ: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [17]:
def get_name_of_manager_by_dlg_id(df, dlg_id):
    
    query = df[(df['dlg_id'] == dlg_id) & (df[COL_INTRODUCE] == True) & (df['role'] == 'manager')]
    if len(query) > 0:
        name = query[COL_NAME].values[0]
        surname = query[COL_SURNAME].values[0]
        patronymic = query[COL_PATRONYMIC].values[0]
        full_name = ' '.join([i for i in filter(None, (name, surname, patronymic))])
        return full_name
    else:
        return 'NoName'

In [18]:
result = df[df['role'] == 'manager'].groupby('dlg_id')[[COL_GREETING, COL_BYE]].any()
result

,greeting,bye
dlg_id,,
0,True,True
1,True,True
2,True,False
3,True,True
4,False,True
5,False,True


In [19]:
for dlg_id in result.index:
    
    manager_name = get_name_of_manager_by_dlg_id(df, dlg_id)
    
    if result.loc[dlg_id, COL_GREETING] and result.loc[dlg_id, COL_BYE]:
        answer =  'выполнил'
    else:
        answer = 'не выполнил'
    print(f'В диалоге №{dlg_id} менеджер {manager_name} требование {answer}')

В диалоге №0 менеджер ангелина требование выполнил
В диалоге №1 менеджер ангелина требование выполнил
В диалоге №2 менеджер ангелина требование не выполнил
В диалоге №3 менеджер максим требование выполнил
В диалоге №4 менеджер NoName требование не выполнил
В диалоге №5 менеджер анастасия требование не выполнил


# ПРОВЕРКА ТРЕБОВАНИЯ: «В каждом диалоге обязательно необходимо поздороваться, представиться и попрощаться с клиентом»

In [20]:
result = df[df['role'] == 'manager'].groupby('dlg_id')[[COL_GREETING, COL_INTRODUCE, COL_BYE]].any()
result

,greeting,introduce,bye
dlg_id,,,
0,True,True,True
1,True,True,True
2,True,True,False
3,True,True,True
4,False,False,True
5,False,True,True


In [21]:
for dlg_id in result.index:
    
    manager_name = get_name_of_manager_by_dlg_id(df, dlg_id)
    
    if result.loc[dlg_id, COL_GREETING] and result.loc[dlg_id, COL_BYE] and result.loc[dlg_id, COL_INTRODUCE]:
        answer =  'выполнил'
    else:
        answer = 'не выполнил'
    print(f'В диалоге №{dlg_id} менеджер {manager_name} требование {answer}')

В диалоге №0 менеджер ангелина требование выполнил
В диалоге №1 менеджер ангелина требование выполнил
В диалоге №2 менеджер ангелина требование не выполнил
В диалоге №3 менеджер максим требование выполнил
В диалоге №4 менеджер NoName требование не выполнил
В диалоге №5 менеджер анастасия требование не выполнил


# ПОЛНЫЙ СПИСОК НАЗВАНИЙ КОМПАНИЙ, КОТОРЫЕ УПОМИНАЮТСЯ В ДИАЛОГАХ

In [22]:
df

,dlg_id,line_n,role,text,company_name,name,surname,patronymic,introduce,greeting,bye
0,0,0,client,алло,None,None,None,None,False,False,False
1,0,1,manager,алло здравствуйте,None,None,None,None,False,True,False
2,0,2,client,добрый день,None,None,None,None,False,True,False
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,компания диджитал бизнес,ангелина,None,None,True,False,False
4,0,4,client,ага,None,None,None,None,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,по поводу виджетов и с ними уже обсудите конкр...,None,None,None,None,False,False,False
476,5,139,manager,все я вам высылаю счет и с вами на связи если ...,None,None,None,None,False,False,False
477,5,140,client,спасибо спасибо,None,None,None,None,False,False,False
478,5,141,client,да да тогда созвонимся ага спасибо вам давайте,None,None,None,None,False,False,False


In [23]:
df[pd.notna(df[COL_COMPANY_NAME])][['dlg_id', COL_COMPANY_NAME]]

,dlg_id,company_name
3,0,компания диджитал бизнес
111,1,компания диджитал бизнес
167,2,компания диджитал бизнес
251,3,компания китобизнес
273,3,компания интеграции
